In [8]:
# Formula for laminar volumetric flow
%pip install -U sympy 

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Importing Packages

In [9]:
import sympy as sp
from sympy.solvers import solve
from sympy import Symbol
import math


# Defining Variables

In [10]:
# DEFINE PARAMETERS
l_tube = 0.2 # [m]
h_water_initial = 0.1 # [m]
h_water_final = 0.02 # [m]
d_tube = 7.94 * 10**-3 # [m]
v_tank = 0
area_tank = 0.32 * 0.26 # [m^2]
t_final = 0 # [s]

# Area of Tube, use only 1:
# 1. Without t-joint
area_tube = math.pi * (d_tube / 2)**2 # [m^2]
# 2. With t-joint
# area_tube_t_joint = 2 * area_tube # [m^2]


## DEFINE CONSTANTS
density = 1000 # [kg/m^3]
viscosity = 8.9 * 10**-4 # [Pa*s]
g = 9.81 # [m/s^2]
e = 0.0025 *(10**-3) #[m]

## DEFINE MODELLING VARIABLES
t_interval = 0.1 # [s]
t_drain = 0 # [s]
h_water = h_water_initial # [m]
initial = True

In [11]:
# HELPER FUNCTIONS
# def ht(v_tube, t):
#     ht = 0.1 - (area_tube / area_tank) * v_tube * t
#     return ht

def reynolds(velocity, length):
    return (density * velocity * length / viscosity)

# CALCULATION - INITIAL CONDITIONS
v_tube = Symbol('v_tube')
v_tube_sol = solve(
    (0.5 * density * ((area_tube / area_tank)**2) * (v_tube**2))
    + (density * g * h_water_initial)
    - (0.5*density*(v_tube**2))
    - ((density * g) * (64 / ((density * v_tube * d_tube) / viscosity)) * (l_tube * v_tube**2 / (2*g*d_tube)) ),
    v_tube)[1]
    

In [14]:
def reynolds(velocity, length):
    return (density * velocity * length / viscosity)

while(h_water > h_water_final):

    h_water -= (area_tube / area_tank) * v_tube_sol * t_interval

    # the imicit equation
    v_tube = Symbol('v_tube')
    f =  Symbol('f')
    cur_reynolds = reynolds(v_tube_sol, d_tube)
    print(cur_reynolds)

    if (cur_reynolds < 2300):
        v_tube_sol= solve(
        (0.5 * density * ((area_tube / area_tank)**2) * (v_tube**2))
        + (density * g * h_water)
        - (0.5*density*(v_tube**2))
        - ((density * g) * (64 / ((density * v_tube * d_tube) / viscosity)) * (l_tube * v_tube**2 / (2*g*d_tube)) ),
        v_tube)[1]
    else:
        friction = solve((1/sp.sqrt(f)) + 2*sp.log((e/d_tube)/3.7)+(2.51/cur_reynolds*sp.sqrt(f)), f)
        v_tube_sol= solve(
        (0.5 * density * ((area_tube / area_tank)**2) * (v_tube**2))
        + (density * g * h_water)
        - (0.5*density*(v_tube**2))
        - ((density * g) * (64 / ((density * v_tube * d_tube) / viscosity)) * (l_tube * v_tube**2 / (2*g*d_tube)) ),
        v_tube)[1]


    print(h_water)

    t_drain += t_interval




    

TypeError: cannot determine truth value of Relational

In [ ]:

print(t_drain / 60)
    


2.4283333333332666
